In [0]:
from functools import reduce
from typing import Dict, List, Any
from datetime import datetime
from datetime import timedelta 
import pkg_resources
import pyspark.sql
import os, glob
import pyspark.sql.functions as F
from pyspark.sql.types import ArrayType, StringType, MapType, StructField, StructType, FloatType
import pyspark.sql.types as T
import importlib
import logging
import pkgutil
import json


In [0]:
%run ../../functions/Functions_delta

Out[3]: <function __main__.reportdate_shifttime(reportdate, time)>

In [0]:
input_file =dbutils.widgets.get("input_file")

#get input parameters
import json
with open(input_file, 'r') as file:
  data = file.read()
jsonObject = json.loads(data)

inputs: Dict[str, Dict[str, Any]] = jsonObject["jointransform"]
query= inputs["query"]
business_key= inputs["bussiness_keys"]
sort_key= inputs["sort_key"]   #26 Nov: Sort_key is mandatory for avoiding duplicates
primary_key=inputs["primary_key"]
non_align_columns=inputs["non_align_columns"]
 
loadstrategy = inputs["loadstrategy"]
inputTable=inputs["inputTable"]
outputTable=inputs["outputTable"]
outputExtract=inputs["outputExtract"]

adlsPath=inputs["adlsPath"]
adlsfolder=inputs["adlsfolder"]
hour_partition=inputs["hour_partition"]
date_column= "current_date" if not inputs["date_column"].strip() else inputs["date_column"].strip()

trans_name= "jointransform"

In [0]:
#input parameters required for sql connections are getting from create_sql_dataframe function, just sqlconnection credential file need to be define in jobinput json file.
sqlconnectionFile = "/dbfs/mnt/deltajobinputs/"+ inputs["sqlconnectionFile"]


In [0]:
sources: Dict[str, Dict[str, Any]] =inputs["sources"]
if len(sources)!=0:
  input_s: Dict[str, pyspark.sql.DataFrame] = extract(sources)
  for key in input_s:
     input_s[key].registerTempTable(key)

In [0]:
#updated stroed proc logic for date loop: provide the start date and end date in jobinput.json file and python datetime format of your date column
#note: query doesnot have any entire null column, spark won't accept it.
ReportDate = datetime.strptime(inputs["start_dt"]["value"], inputs["start_dt"]["python_format"])
end_Dt= datetime.strptime(inputs["end_dt"]["value"], inputs["end_dt"]["python_format"])
calc_level = 3
reports_input = "abfss://deltajobinputs@dlenterpriseanalytics.dfs.core.windows.net/Reports_input/"+ inputs["reports_input"]

if ReportDate != end_Dt:
  while ReportDate <= end_Dt:
    for step in inputs['SP']:
      temptable = inputs['SP'][step]["temptable"] #name for temporary table from input file->SP->step1->temp
      stepdf = spark.sql(inputs['SP'][step]["query"].format(ReportDate))
      stepdf.registerTempTable(temptable)  

    report_Measure_calculation(stepdf, calc_level, reports_input )

    final_table = spark.sql("select * from tempreport" )    
    if final_table.count != 0:
      final_table.write.mode("append").saveAsTable("final_table_temp")
    ReportDate = ReportDate + timedelta(days=1)
else:
  for step in inputs['SP']:
      temptable = inputs['SP'][step]["temptable"] #name for temporary table from input file->SP->step1->temp
      stepdf = spark.sql(inputs['SP'][step]["query"].format(ReportDate))
      stepdf.registerTempTable(temptable)  

  report_Measure_calculation(stepdf, calc_level, reports_input )

  final_table = spark.sql("select * from tempreport" )    
  if final_table.count != 0:
    final_table.write.mode("append").saveAsTable("final_table_temp")

In [0]:
%run ../transform_delta

Out[45]: '\nDec 01, 2020: When sqlrefresh true then newly added \'sqlrefresh_query\' parameter can refresh only 7 days prior data from SQL server. \nNote: If sqlrefresh_query = "" or sqlrefresh_query is not provided in inputjson then entire SQL table will be refreshed into ADLS.\n\nDec02, 2020: when sqlrefresh true/false save SQL table in temporary ADLS table.\noverwrite will directly overwrite ADLS and SQL data\nAppend: SQL refresh false: Incoming data will append to ADLS and SQL data\nAppend: SQL refresh true: SQL data will merge into ADLS and then incoming data will merge into ADLS and SQL\nupsert: if sqlrefresh is true or false then SQL table data will be loaded into temp table.\n sqlrefresh: false: incoming data will merge into ADLS and then merge into internal temp(sql refresh data) with name outputDB.outputTable+"_temp". Temp table extract will load into sql\n sqlrefresh: true: incoming data will merge into ADLS and then merge into temp(sql refresh data). Temp table extract will load into sql. After this Temp table extract will merge into ADLS table\n - Temp table will be dropped once load completed.\nthen merge final sql table data into ADLS\n#Dec, 12, 2020: 1. sqlSchema parameter indicates schema name of the table. if not provided or blank then default "dbo" schema will be used. \n 2. truncate parameter is added for truncate in sql table write. It is added incase of overwrite and if truncate option should be \'False\'.\n'

Out[48]: <function __main__.reportdate_shifttime(reportdate, time)>

abfss://deltastage@dlenterpriseanalytics.dfs.core.windows.net/

['year', 'month', 'date']

In [0]:
spark.sql("drop table final_table_temp")

In [0]:
%sql
select * from deltadatamart.fact_calculated_production_measure_wetmillYield

PlantID,ReportDateID,CalcCategory,CalcFrequency,CalcName,CalcValue,UoM,CalcDesc,TextInput,year,month,date,DateInserted,DateUpdated
220,20210210,Wetmill Yield Estimates,Daily,A5_corn_moisture,14.486529,Null,Grainsmart,Null,2021,2,10,2021-02-24T16:38:23.638+0000,2021-02-24T16:38:23.638+0000
220,20210210,Wetmill Yield Estimates,Daily,A7_corn_moisture,14.378844,Null,Grainsmart,Null,2021,2,10,2021-02-24T16:38:23.638+0000,2021-02-24T16:38:23.638+0000
220,20210210,Wetmill Yield Estimates,Daily,Fiber_YieldEstimate,0.0,Null,=[WetFiber_lbs_DB]/(sum of (all corn grind types in db)),Null,2021,2,10,2021-02-24T16:38:23.638+0000,2021-02-24T16:38:23.638+0000
220,20210210,Wetmill Yield Estimates,Daily,Germ_YieldEstimate,0.0,Null,=[Germ_lbs_DB]/(sum of (all corn grind types in db)),Null,2021,2,10,2021-02-24T16:38:23.638+0000,2021-02-24T16:38:23.638+0000
220,20210210,Wetmill Yield Estimates,Daily,Germ_lbs_CB,1325200.0,Null,SAP 13600011AO,Null,2021,2,10,2021-02-24T16:38:23.638+0000,2021-02-24T16:38:23.638+0000
220,20210210,Wetmill Yield Estimates,Daily,Germ_lbs_DB,1298696.0,Null,=[Germ_lbs_CB]*(1-[Germ_moisture]),Null,2021,2,10,2021-02-24T16:38:23.638+0000,2021-02-24T16:38:23.638+0000
220,20210210,Wetmill Yield Estimates,Daily,Germ_moisture,2.0,Null,https://nkcing.azurewebsites.net/GrindReports.aspx,Null,2021,2,10,2021-02-24T16:38:23.638+0000,2021-02-24T16:38:23.638+0000
220,20210210,Wetmill Yield Estimates,Daily,GlutenMeal_YieldEstimate,0.0,Null,=[GlutenMeal_lbs_DB]/(sum of (all corn grind types in db)),Null,2021,2,10,2021-02-24T16:38:23.638+0000,2021-02-24T16:38:23.638+0000
220,20210210,Wetmill Yield Estimates,Daily,GlutenMeal_lbs_CB,357600.0,Null,"SAP Code 13895000AO, 13895000AP",Null,2021,2,10,2021-02-24T16:38:23.638+0000,2021-02-24T16:38:23.638+0000
220,20210210,Wetmill Yield Estimates,Daily,GlutenMeal_lbs_DB,348624.24,Null,=[GlutenMeal_lbs_CB]*(1-[GlutenMeal_moisture]),Null,2021,2,10,2021-02-24T16:38:23.638+0000,2021-02-24T16:38:23.638+0000
